In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [10]:
# read in data from July, 28th, 2012
trajectories28 = pd.read_csv("TrajDataV2_20120728.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
# read in data from July, 29th, 2012
trajectories29 = pd.read_csv("TrajDataV2_20120729.txt",  
                             delimiter="\t", 
                             names=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'),
                             usecols=('flight', 'date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude')
                            )
trajectories28.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
0,845.0,28.0,0.0,1.0,470.31621,39000.0,40.692500,-74.168667
1,845.0,28.0,0.0,2.0,470.31621,39000.0,40.784686,-74.019193
2,845.0,28.0,0.0,3.0,470.31621,39000.0,40.877281,-73.868705
3,845.0,28.0,0.0,4.0,470.31621,39000.0,40.969822,-73.717979
4,845.0,28.0,0.0,5.0,470.31621,39000.0,41.062186,-73.566848


In [11]:
# concatenate data from both days
#trajectories = pd.concat([trajectories28, trajectories29])
trajectories = trajectories29

In [12]:
# add consecutive flight index to the data
flightNames = trajectories['flight'].unique()
trajectories['flightIndex'] = trajectories['flight'].map(lambda x: np.where(flightNames==x)[0][0])
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude,flightIndex
0,805.0,29.0,0.0,2.0,465.28117,33000.0,40.692500,-74.168667,0
1,805.0,29.0,0.0,3.0,465.28117,33000.0,40.785863,-74.013012,0
2,805.0,29.0,0.0,4.0,465.28117,33000.0,40.878762,-73.856473,0
3,805.0,29.0,0.0,5.0,465.28117,33000.0,40.971273,-73.699485,0
4,805.0,29.0,0.0,6.0,465.28117,33000.0,41.063590,-73.542087,0


In [13]:
# set consecutive flight index as dataset index
trajectories = trajectories.set_index('flightIndex')
trajectories.head()

,flight,date,wind,time,speed,altitude,latitude,longitude
flightIndex,,,,,,,,
0,805.0,29.0,0.0,2.0,465.28117,33000.0,40.692500,-74.168667
0,805.0,29.0,0.0,3.0,465.28117,33000.0,40.785863,-74.013012
0,805.0,29.0,0.0,4.0,465.28117,33000.0,40.878762,-73.856473
0,805.0,29.0,0.0,5.0,465.28117,33000.0,40.971273,-73.699485
0,805.0,29.0,0.0,6.0,465.28117,33000.0,41.063590,-73.542087


In [86]:
trajectories.time.plot(marker='o', linestyle='o')
plt.show()

In [83]:
def plotTrajectories(trajectories):
    # Create a figure of size (i.e. pretty big)
    fig = plt.figure(figsize=(20,10))

    # Create a map, using the Gall–Peters projection, 
    map = Basemap(projection='gall', 
                  # with low resolution,
                  resolution = 'l', 
                  # And threshold 100000
                  area_thresh = 100000.0,
                  # Centered at 0,0 (i.e null island)
                  lat_0=0, lon_0=0)

    # Draw the coastlines on the map
    map.drawcoastlines()

    # Draw country borders on the map
    map.drawcountries()

    # Fill the land with grey
    map.fillcontinents(color = '#888888')

    # Draw the map boundaries
    map.drawmapboundary(fill_color='#f4f4f4')

    # Define our longitude and latitude points
    # We have to use .values because of a wierd bug when passing pandas data
    # to basemap.
    x,y = map(trajectories['longitude'].values, trajectories['latitude'].values)

    # Plot them using round markers of size 6
    map.plot(x, y, 'b', markersize=6)

    # Show the map
    plt.show()

In [84]:
plotTrajectories(trajectories[trajectories.index < 100])

In [26]:
def distance(lat1, lon1, lat2, lon2, R=6367):
    """Get the distance on a great circle between to trajectory points in kilometers
    
    Arguments:
    R: Radius in kilometers
    lat1: latitude of first point in degrees
    lon1: longitude of the first point in degrees
    lat2: latitude of the second point in degrees
    lon2: longitude of the second point in degrees
    
    """
    Lat0 = np.radians(lat1)
    Latf = np.radians(lat2)
    Lon0 = np.radians(lon1)
    Lonf = np.radians(lon2)

    return R * np.arccos(np.sin(Lat0) * np.sin(Latf) + np.cos(Lonf-Lon0)*np.cos(Lat0)*np.cos(Latf))

In [35]:
# test
i1 = 4
i2 = 7
lat1 = trajectories['latitude'].iloc[i1]
lat2 = trajectories['latitude'].iloc[i2]
lon1 = trajectories['longitude'].iloc[i1]
lon2 = trajectories['longitude'].iloc[i2]
distance(lat1, lon1, lat2, lon2)

49.866839043922198

In [36]:
# constants 
# nautic mile in kilometers
nautic = 1.852

In [37]:
# minimal acceptable distance in kilometers
mindistance = 30 * nautic

In [34]:
# minimal acceptable time difference
mintime = 3

In [38]:
def detectSpatialConflict(lat1, lon1, lat2, lon2, mindistance):
    return distance(lat1, lon1, lat2, lon2) < mindistance

In [ ]:
# detect spatial conflict points
trajectories['spatialConflictPoint'] = trajectories['flight'].map(lambda x: np.where(flightNames==x)[0][0])
trajectories.head()